In [ ]:
%matplotlib inline

from ipyleaflet import Map, ImageOverlay, Polyline
import matplotlib
import matplotlib.cm
from matplotlib import pyplot
import numpy as np
import numpy.lib.recfunctions
import scipy
import scipy.interpolate
import pandas
import platypus.io.logs
import os
import uuid
import glob

In [ ]:
# Import the data from the specified logfile
log_filename = '../logs/platypus_20161126_161922.txt'
data = platypus.io.logs.load(log_filename)

# Define useful access variables.
pose = data['pose']
position = pose[['latitude', 'longitude']]

# Print the available sensors and channels for this logfile.
print "Available sensors/channels:"
for s in data.keys():
    if s == 'pose' or s == 'BATTERY':
        continue
    for c in data[s].dtypes.keys():
        print "  {:s}, {:s}".format(s, str(c))

In [ ]:
# Select the sensor and the name of the channel for that sensor.
sensor_name = 'HDS_DEPTH'
sensor_channel = 1
sensor_units = 'meters (m)'

In [ ]:
# Extract the pose timing and the sensor data of interest.
pose_times = pose.index.values.astype(np.float64)

sensor = data[sensor_name]
sensor_times = sensor.index.values.astype(np.float64)

# Linearly interpolate the position of the sensor at every sample.
sensor_pose_interpolator = scipy.interpolate.interp1d(pose_times, pose[['latitude', 'longitude']],
                                                      axis=0, bounds_error=False)

# Add the position information back to the sensor data.
sensor = sensor.join(pandas.DataFrame(sensor_pose_interpolator(sensor_times), sensor.index,
                                      columns=('latitude', 'longitude')))

# Remove columns that have NaN values (no pose information).
sensor_valid = np.all(np.isfinite(sensor), axis=1)
sensor = sensor[sensor_valid]

In [ ]:
# Create a trail of the vehicle's path on the map.
pl = Polyline(locations=position.as_matrix().tolist())
pl.fill_opacity = 0.0
pl.weight = 2

In [ ]:
# Add a data overlay for the map
data_padding = [0.0001, 0.0001]   # degrees lat/lon
data_resolution = [0.00001, 0.00001] # degrees lat/lon
data_interpolation_radius = 0.00005 # degrees lat/lon
data_bounds = [(position.min() - data_padding).tolist(),
               (position.max() + data_padding).tolist()]

# Create a rectangular grid of overlay points.
data_xv, data_yv = np.meshgrid(
    np.arange(data_bounds[1][0], data_bounds[0][0], -data_resolution[0]),
    np.arange(data_bounds[0][1], data_bounds[1][1], data_resolution[1])
)
data_shape = data_xv.shape
data_xy = np.vstack([data_xv.ravel(), data_yv.ravel()]).T

# Create a radial-basis interpolator over the sensor dataset
# Then, query it at each point of the rectangular grid.
from sklearn.neighbors import RadiusNeighborsClassifier
data_estimator = RadiusNeighborsClassifier(radius=data_interpolation_radius, outlier_label=np.nan)
data_estimator.fit(sensor[['latitude','longitude']], sensor[sensor_channel])
data_zv = data_estimator.predict(data_xy)
data_zv = data_zv.reshape(data_shape).T

# Normalize data from [0, 1)
data_max = data_zv[np.isfinite(data_zv)].max()
data_min = data_zv[np.isfinite(data_zv)].min()
data_zv = (data_zv - data_min) / (data_max - data_min)

# Update a color map only at the points that have valid values.
data_rgb = np.zeros((data_shape[0], data_shape[1], 4), dtype=np.uint8)
data_rgb = matplotlib.cm.jet(data_zv)*255.0
data_rgb[:,:,3] = 255 * np.isfinite(data_zv)

# Remove any old image files.
old_png_files = glob.glob('./*.png')
for old_png_file in old_png_files:
    os.remove(old_png_file)

png_filename = './platypus_data_{:s}.png'.format(uuid.uuid4())
scipy.misc.imsave(png_filename, data_rgb)

# Create image overlay that references generated image.
io = ImageOverlay(url=png_filename, bounds=data_bounds)

In [ ]:
# Create a map centered on this data log.
center = [pose['latitude'].median(), pose['longitude'].median()]
zoom = 18
m = Map(center=center, zoom=zoom, height='600px')
m += io # Add image overlay
m += pl # Add vehicle trail

# Make a figure and axes with dimensions as desired.
fig = pyplot.figure(figsize=(15, 3))
ax1 = fig.add_axes([0.05, 0.80, 0.9, 0.15])

# Set the colormap and norm to correspond to the data for which
# the colorbar will be used.
cmap = matplotlib.cm.jet
norm = matplotlib.colors.Normalize(vmin=data_min, vmax=data_max)
cb1 = matplotlib.colorbar.ColorbarBase(ax1, cmap=cmap,
                                       norm=norm,
                                       orientation='horizontal')
cb1.set_label(sensor_units)

pyplot.show()
m